### Imports

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd

### Data Clean

In [3]:
data_path = Path(r"C:\Users\conne\Documents\Csci-Capstone\Data\CIC-IDS2017")
csv_files = list(data_path.glob("*.csv"))

df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)
print(f"Loaded {len(csv_files)} files with shape {df.shape}")


df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].str.strip()

df.replace([np.inf, -np.inf], np.nan, inplace=True)

df = df.loc[:, df.isna().mean() < 0.3]

numeric_cols = df.select_dtypes(include="number").columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

df.drop_duplicates(inplace=True)

drop_cols = [
    "fwd_header_length.1",
    "fwd_avg_bytes/bulk","fwd_avg_packets/bulk","fwd_avg_bulk_rate",
    "bwd_avg_bytes/bulk","bwd_avg_packets/bulk","bwd_avg_bulk_rate"
]
df = df.drop(columns=drop_cols, errors="ignore")

Loaded 8 files with shape (2830743, 79)


C:\Users\conne\AppData\Local\Temp\ipykernel_4820\996074344.py:10: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  for col in df.select_dtypes(include="object").columns:


In [4]:
# -------------------------------
# Layered IDS: Isolation Forest (Anomaly Scoring Only)
# -------------------------------

import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import joblib

# -------------------------------
# 1️⃣ Extract numeric features
# -------------------------------
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remove label if present
if 'label' in numeric_features:
    numeric_features.remove('label')

X_unsup = df[numeric_features]

# -------------------------------
# 2️⃣ Remove highly correlated features
# -------------------------------
corr_matrix = X_unsup.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [col for col in upper.columns if any(upper[col] > 0.9)]
X_unsup_clean = X_unsup.drop(columns=to_drop)

# -------------------------------
# 3️⃣ Scale features
# -------------------------------
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X_unsup_clean)

# -------------------------------
# 4️⃣ Train / validation split
# (labels ONLY for offline evaluation)
# -------------------------------
y_eval = df['label'].apply(lambda x: 0 if x.lower() == 'benign' else 1)

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled,
    y_eval,
    test_size=0.2,
    stratify=y_eval,
    random_state=42
)

# -------------------------------
# 5️⃣ Train Isolation Forest
# -------------------------------
iso = IsolationForest(
    n_estimators=200,
    max_samples='auto',
    contamination=0.05,
    random_state=42
)

iso.fit(X_train)

# -------------------------------
# 6️⃣ Generate anomaly scores
# (higher = more anomalous)
# -------------------------------
train_scores = -iso.score_samples(X_train)
val_scores = -iso.score_samples(X_val)

# -------------------------------
# 7️⃣ Normalize scores for ensemble fusion
# -------------------------------
score_scaler = MinMaxScaler()

train_scores_norm = score_scaler.fit_transform(
    train_scores.reshape(-1, 1)
).ravel()

val_scores_norm = score_scaler.transform(
    val_scores.reshape(-1, 1)
).ravel()

# -------------------------------
# 8️⃣ Define risk tiers (NO hard classification)
# -------------------------------
LOW_RISK = np.percentile(train_scores_norm, 75)
MEDIUM_RISK = np.percentile(train_scores_norm, 90)
HIGH_RISK = np.percentile(train_scores_norm, 97)

def iso_risk_label(score):
    if score >= HIGH_RISK:
        return "high"
    elif score >= MEDIUM_RISK:
        return "medium"
    elif score >= LOW_RISK:
        return "low"
    else:
        return "normal"

risk_labels_val = [iso_risk_label(s) for s in val_scores_norm]

# -------------------------------
# 9️⃣ Offline evaluation only
# -------------------------------
print("Isolation Forest ROC-AUC:",
      roc_auc_score(y_val, val_scores_norm))

# -------------------------------
# 🔟 Save artifacts for downstream layers
# -------------------------------
joblib.dump(iso, "iso_forest_model.pkl")
joblib.dump(feature_scaler, "iso_feature_scaler.pkl")
joblib.dump(score_scaler, "iso_score_scaler.pkl")
joblib.dump(to_drop, "iso_dropped_features.pkl")



Isolation Forest ROC-AUC: 0.6518532530579756


['iso_dropped_features.pkl']